# Example queries on COVID-19 Knowledge Graph
[Work in progress]

In [1]:
import os
import time
import pandas as pd
from py2neo import Graph

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
NEO4J_HOME = os.getenv('NEO4J_HOME')
print(NEO4J_HOME)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3


Start neo4j database if it is not already running or restart database to load the latest version of the database.

In [4]:
!"$NEO4J_HOME"/bin/neo4j start

Directories in use:
  home:         /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3
  config:       /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3/conf
  logs:         /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3/logs
  plugins:      /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3/plugins
  import:       /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3/import
  data:         /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a2

Wait until database is started up

In [5]:
time.sleep(15)

In [6]:
graph = Graph("bolt://localhost:7687/", password="neo4jbinder")

### List Organisms in KG

In [7]:
query = """
MATCH (p:Organism)
RETURN p.name as name, p.scientificName as scientificName, p.id as taxonomy
"""
graph.run(query).to_data_frame()

,name,scientificName,taxonomy
0,SARS-CoV-2,Severe acute respiratory syndrome coronavirus 2,taxonomy:2697049
1,MERS-CoV,Middle East respiratory syndrome-related coron...,taxonomy:1335626
2,SARS-CoV,Severe acute respiratory syndrome-related coro...,taxonomy:694009
3,human,Homo sapiens,taxonomy:9606
4,intermediate horseshoe bat,Rhinolophus affinis,taxonomy:59477
5,horseshoe bat,Rhinolophus,taxonomy:49442
6,Malayan pangolin,Manis javanica,taxonomy:9974
7,palm civet,Paradoxurus,taxonomy:71116
8,carnivores,Canidae,taxonomy:9608


### List Coronavirus Outbreaks

In [8]:
query = """
MATCH (p:Organism)-[:CAUSES]->(o:Outbreak)
RETURN p.name as name, p.scientificName as scientificName, p.id as taxonomy, o.id as outbreak, o.startDate as startDate
"""
graph.run(query).to_data_frame()

,name,scientificName,taxonomy,outbreak,startDate
0,SARS-CoV-2,Severe acute respiratory syndrome coronavirus 2,taxonomy:2697049,COVID-19,2019
1,MERS-CoV,Middle East respiratory syndrome-related coron...,taxonomy:1335626,MERS,2012
2,SARS-CoV,Severe acute respiratory syndrome-related coro...,taxonomy:694009,SARS,2003


### List Strains that are mentions in PubMed Central Articles

In [9]:
query = """
MATCH (p:Publication)-[:MENTIONS]->(s:Strain)<-[:CARRIES]-(o:Organism)
RETURN p.id as pmc, s.name as name, s.collectionDate  as collectionDate, o.name as host, s.id as host_id
ORDER by s.collectionDate
"""
graph.run(query).to_data_frame()
# TODO where do the 2013 bat strains come from??

,pmc,name,collectionDate,host,host_id
0,pmc:PMC7095418,bat/Yunnan/RaTG13/2013,2013-07-24,intermediate horseshoe bat,https://www.gisaid.org/EPI_ISL_402131
1,pmc:PMC7106073,bat/Yunnan/RaTG13/2013,2013-07-24,intermediate horseshoe bat,https://www.gisaid.org/EPI_ISL_402131
2,pmc:PMC7067954,Wuhan-Hu-1,2019-12-13,human,ncbiprotein:NC_045512
3,pmc:PMC7067954,Wuhan-Hu-1,2019-12-13,human,ncbiprotein:NC_045512
4,pmc:PMC7102578,Wuhan-Hu-1,2019-12-13,human,ncbiprotein:NC_045512
5,pmc:PMC7102578,Wuhan-Hu-1,2019-12-13,human,ncbiprotein:NC_045512
6,pmc:PMC7060195,Wuhan-Hu-1,2019-12-13,human,ncbiprotein:NC_045512
7,pmc:PMC7060195,Wuhan-Hu-1,2019-12-13,human,ncbiprotein:NC_045512
8,pmc:PMC7062204,Wuhan-Hu-1,2019-12-13,human,ncbiprotein:NC_045512
9,pmc:PMC7062204,Wuhan-Hu-1,2019-12-13,human,ncbiprotein:NC_045512


### List Gene and Protein information for Reference Genome

In [10]:
query = """
MATCH (s:Strain)-[:HAS]->(g:Gene)-[:ENCODES]->(p:Protein)
RETURN s.id as referenceGenome, s.name as name, s.collectionDate  as collectionDate, 
       g.name as gene, g.id as geneId, p.name as protein, p.id as protein_id 
ORDER by s.collectionDate
"""
graph.run(query).to_data_frame()

,referenceGenome,name,collectionDate,gene,geneId,protein,protein_id
0,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-13,ORF1ab,ncbigene:43740578,nsp11,ncbiprotein:YP_009725312
1,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-13,ORF1ab,ncbigene:43740578,nsp4,ncbiprotein:YP_009725300
2,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-13,ORF1ab,ncbigene:43740578,nsp2,ncbiprotein:YP_009725298
3,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-13,ORF1ab,ncbigene:43740578,helicase,ncbiprotein:YP_009725308
4,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-13,ORF1ab,ncbigene:43740578,nsp10,ncbiprotein:YP_009725306
5,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-13,ORF1ab,ncbigene:43740578,2'-O-ribose methyltransferase,ncbiprotein:YP_009725311
6,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-13,ORF1ab,ncbigene:43740578,nsp7,ncbiprotein:YP_009725303
7,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-13,ORF1ab,ncbigene:43740578,endoRNAse,ncbiprotein:YP_009725310
8,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-13,ORF1ab,ncbigene:43740578,ORF1a polyprotein,ncbiprotein:YP_009725295
9,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-13,ORF1ab,ncbigene:43740578,RNA-dependent RNA polymerase,ncbiprotein:YP_009725307


In [11]:
!"$NEO4J_HOME"/bin/neo4j stop

Stopping Neo4j....... stopped


## Old queries need to be updated

### List person demographics and strain information for California

In [12]:
# query = """
# MATCH (a:Admin1)<-[:LOCATED_IN]-(p:Person)-[:CARRIES]->(s:Strain)
# WHERE a.name = 'California'
# RETURN p.age as age, p.sex as sex, p.exposure_location as exposure_location, s.name as strain, s.clade as clade
# """
# graph.run(query).to_data_frame()

#### Same query using parameterized Cypher
Parameters to Cypher queries can be passed as key-value arguments. Parameters in Cypher are named and are wrapped in curly braces.

In [13]:
# admin1 = 'California'

# query = """
# MATCH (a:Admin1{name: {admin1}})<-[:LOCATED_IN]-(p:Person)-[:CARRIES]->(s:Strain)
# RETURN p.age as age, p.sex as sex, p.exposure_location as exposure_location, 
#        s.name as strain, s.clade as clade, s.date as date
# ORDER BY s.date
# """
# graph.run(query, admin1=admin1).to_data_frame().head(100)

### Where did clade A originate?

In [14]:
# clade = 'A'

# query = """
# MATCH (s:Strain)--(a:Country)
# WHERE s.clade STARTS WITH {clade}
# RETURN s.clade as clade, s.name, s.date, a.name
# ORDER BY s.date
# """
# graph.run(query, clade=clade).to_data_frame().head(100)

### Find persons that imported the virus from another location

In [15]:
# query = """
# MATCH (c:Admin1)<-[:LOCATED_IN]-(p:Person)-[:CARRIES]->(s:Strain)
# WHERE c.name <> p.exposure_location
# RETURN c.name as `state/province`, p.age as age, p.sex as sex, p.exposure_location as exposure_location, 
#        s.name as strain, s.clade as clade
# ORDER BY p.exposure_location
# """
# graph.run(query).to_data_frame()

### Strains in Sydney

In [16]:
# city = 'Sydney'

# query = """
# MATCH (c:City{name: {city}})<-[:LOCATED_IN]-(p:Person)-[:CARRIES]->(s:Strain)
# RETURN c.name as city, s.name as strain, s.clade as clade, p.exposure_location, s.date as date
# ORDER BY s.date
# """
# graph.run(query, city=city).to_data_frame()